Stackoverflow Questions tagging with multiple labels.<br>
Dataset - Hackerearth Deeplearning Challenege #4 Dataset.<br>
Architecture - Nvidia GPU 

In [1]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [27]:
import tensorflow as tf
from tensorflow.python.framework import ops 

from collections import Counter 
import pandas as pd
import numpy as np
import string
import re
from _operator import index
import time
import os

In [3]:
def clean_text(text):
    global EMPTY
    EMPTY = ''
    
    if not isinstance(text, str):
        return text
    
    text = re.sub('<pre><code>.*?</code></pre>',EMPTY,text)
    text = re.sub('<a[^>]+>(.*)</a>', replace_link, text)
    return re.sub('<[^>]+>', EMPTY, text)

def replace_link(match):
    return EMPTY if re.match('[a-z]+://',match.group(1)) else match.group(1)

def process_data(filename):
    df = pd.read_csv(filename)
    
    DATA = pd.DataFrame(columns=['article','tags'])
    DATA['article'] = df['article'].apply(clean_text).str.lower() 
    DATA['article'] = DATA['article'].apply(lambda x: x.replace('"','').replace("\n","").replace("\t","").replace("\r",""))
    DATA['tags'] = df['tags'].values
    DATA = DATA.dropna()
    
    return DATA

In [4]:
start = time.time()
filename = './new_dataset/train.csv'
train_data = process_data(filename)
end = time.time()

elapsed = end - start 
print("Time to read dataset"+str(elapsed))
    

Time to read dataset60.38761496543884


In [5]:
train_X = train_data['article']
train_Y = train_data['tags']

In [6]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import *

from sklearn.manifold import TSNE

Using TensorFlow backend.


In [7]:
def process_text(text):
    text = text.translate(string.punctuation)
    
    text = text.lower().split()
    
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text

In [8]:
# Uncomment when required to process text 
# train_X = train_X.map(lambda x: process_text(x))
# path = r'C:\Users\adilf\Deeplearning Projects\hackerearth#4\process-dataset'
# train_X.to_csv(os.path.join(path,r'train_X.csv'))

In [28]:
path = r'C:\Users\adilf\Deeplearning Projects\hackerearth#4\process-dataset'
train_X.to_csv(os.path.join(path,r'train_X.csv'))

In [32]:
train_X[0]
    

'way find sort request flash applic send server i tri see inform client send server use chrome inspect element show noth go on but sure communic go on websit interest everi time make move somehow send server may anoth oppon end game send game server sure till now see imag upload game thank help p s tri use wireshark captur packet way see communic do : ping chesscub com realiz ip - address : 78 47 2 115than listen packag address ip addr 78 47 2 115but thing see lot tcp http packag http packag send png imag avatar user chat there peopl constant speak see that understand go anoth address idea found it problem watch traffic net comput much it know limit it'

In [33]:
# Process labels
def process_labels(text):
    text = text.replace("|"," ")
    return text

In [34]:
# Uncomment when required to process labels
# train_Y = train_Y.map(lambda x: process_labels(x))
# path = r'C:\Users\adilf\Deeplearning Projects\hackerearth#4\process-dataset'
# train_Y.to_csv(os.path.join(path,r'train_Y.csv'))

In [41]:
path = r'C:\Users\adilf\Deeplearning Projects\hackerearth#4\process-dataset'
train_Y.to_csv(os.path.join(path,r'train_Y.csv'))

In [53]:
def get_vocab_size(labels):
    tags = []
    
    for label in labels.values:
        tag = label.split(" ")
        tags.extend(tag)
    tags = set(tags)
    print("Vocab Size - " + str(len(tags)))

In [54]:
get_vocab_size(train_Y)

Vocab Size - 36321


In [58]:
### Create X sequence
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_X)
sequences = tokenizer.texts_to_sequences(train_X)
data_X = pad_sequences(sequences, maxlen=50)

In [59]:
### Create Y sequence
vocabulary_size = 36330
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_Y)
sequences = tokenizer.texts_to_sequences(train_Y)
data_Y = pad_sequences(sequences)

In [64]:
data_X[0]

array([1048,  333, 2652, 1374,   22,  740,  271,  333, 1048, 1648, 2652,
       1374,   22,  362,  170,  647, 1454,   68,  271,   68,  271,  270,
        512,   55, 2812,   25, 1668,  505, 1033, 1117, 4282,  170,  280,
        416,  518,  275,  333,  295,  229,   93,   80, 1650, 3176,  296,
        783,  577,   93,  106,  707,   93])

In [65]:
len(data_X[0])

50

In [66]:
data_Y[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  199,   88,  495, 1936, 2161])